In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from numbers import Number
from ta import momentum as mo
from ta import volume as vo
from ta import trend as tr

In [2]:
hist_data_path = 'data/tsla_1.5y.csv'
hist_data = pd.read_csv(hist_data_path)
hist_data

,date,open,high,low,close,adj_close,volume
0,2020-03-02,142.251999,148.738007,137.334000,148.723999,148.723999,100975000
1,2020-03-03,161.000000,161.395996,143.222000,149.102005,149.102005,128920000
2,2020-03-04,152.792007,153.304001,144.945999,149.899994,149.899994,75245000
3,2020-03-05,144.753998,149.149994,143.613998,144.908005,144.908005,54263500
4,2020-03-06,138.000000,141.399994,136.854004,140.695999,140.695999,63314500
...,...,...,...,...,...,...,...
387,2021-09-13,740.210022,744.780029,708.849976,743.000000,743.000000,22952500
388,2021-09-14,742.570007,754.469971,736.400024,744.489990,744.489990,18524900
389,2021-09-15,745.000000,756.859985,738.359985,755.830017,755.830017,15357700
390,2021-09-16,752.830017,758.909973,747.609985,756.989990,756.989990,13923400


In [3]:
# ROC de Y em relação a X
def change(x, y):
    if len(x) == len(y):
        change_arr = []
        for i in range(0, len(x)):
            if isinstance(x[i], Number) and isinstance(y[i], Number):
                change_arr.append(((y[i] - x[i]) / x[i]) * 100)
            else:
                change_arr.append('NaN')
        
        return change_arr
    else:
        print(f"Error: x lenght {len(x)}, y lenght {len(y)}")

In [4]:
# print(change([1, 2],[2, 1])) # [100.0, -50.0] Yn > Xn se valor positivo
trend_window = 30
slice_start = 26
slice_plus = 30

roc = mo.roc(hist_data['close'], window=12)
rsi = mo.rsi(hist_data['close'])
stoch = mo.stoch(hist_data['high'], hist_data['low'], hist_data['close'])
ema_w8 = tr.ema_indicator(hist_data['close'], window=8)
ema_w20 = tr.ema_indicator(hist_data['close'], window=20)

sma_w9 = tr.sma_indicator(hist_data['close'], window=9)
sma_w26 = tr.sma_indicator(hist_data['close'], window=26)

roc_close_ema_w8 = change(ema_w8, hist_data['close'])
roc_ema_w8_ema_w20 = change(ema_w20, ema_w8)
roc_close_ema_w20 = change(ema_w20, hist_data['close'])
roc_close_open = change(hist_data['open'], hist_data['close'])

roc_close_sma_w9 = change(sma_w9, hist_data['close'])
roc_sma_w9_sma_w26 = change(sma_w26, sma_w9)
roc_close_sma_w26 = change(sma_w26, hist_data['close'])

trend = mo.roc(hist_data['close'], window=trend_window).map(lambda n : 'HIGH' if n > 0 else 'LOW')

# print(roc[20:-60])
# print(trend[50:-30])

In [5]:
df = pd.DataFrame({
#     'close': np.array(list(hist_data['close'][slice_start:-(trend_window + slice_plus)])),
    'roc': np.array(list(roc[slice_start:-(trend_window + slice_plus)])),
    'rsi': np.array(list(rsi[slice_start:-(trend_window + slice_plus)])),
    'stoch': np.array(list(stoch[slice_start:-(trend_window + slice_plus)])),
#     'ema_w8': np.array(list(ema_w8[slice_start:-(trend_window + slice_plus)])),
#     'ema_w20': np.array(list(ema_w20[slice_start:-(trend_window + slice_plus)])),
    'roc_close_ema_w8': np.array(list(roc_close_ema_w8[slice_start:-(trend_window + slice_plus)])),
    'roc_ema_w8_ema_w20': np.array(list(roc_ema_w8_ema_w20[slice_start:-(trend_window + slice_plus)])),
    'roc_close_ema_w20': np.array(list(roc_close_ema_w20[slice_start:-(trend_window + slice_plus)])),
#     'roc_close_open': np.array(list(roc_close_open[slice_start:-(trend_window + slice_plus)])),
    
    'roc_close_sma_w9': np.array(list(roc_close_sma_w9[slice_start:-(trend_window + slice_plus)])),
    'roc_sma_w9_sma_w26': np.array(list(roc_sma_w9_sma_w26[slice_start:-(trend_window + slice_plus)])),
    'roc_close_sma_w26': np.array(list(roc_close_sma_w26[slice_start:-(trend_window + slice_plus)])),
    'trend': np.array(list(trend[(slice_start + trend_window):-slice_plus]))
})
df

,roc,rsi,stoch,roc_close_ema_w8,roc_ema_w8_ema_w20,roc_close_ema_w20,roc_close_sma_w9,roc_sma_w9_sma_w26,roc_close_sma_w26,trend
0,27.581690,49.779482,90.534511,7.806122,-3.853406,3.651914,7.977112,-6.415359,1.049992,HIGH
1,26.376379,50.257685,89.540447,6.470676,-2.441565,3.871125,8.156230,-4.653537,3.123140,HIGH
2,13.465345,53.645385,98.676206,8.468114,-0.820203,7.578455,11.485828,-2.181634,9.053615,HIGH
3,20.713959,62.515591,99.489319,17.176606,2.137574,19.681342,22.704484,1.512246,24.560079,HIGH
4,34.408126,67.568949,89.173533,20.354723,5.381429,26.831527,28.800038,5.414769,35.774263,HIGH
...,...,...,...,...,...,...,...,...,...,...
301,-1.170067,50.268135,70.498626,1.375798,-0.394744,0.975624,1.525234,1.285042,2.829876,HIGH
302,3.006016,52.268296,83.237477,1.917547,-0.043512,1.873201,2.289846,1.404542,3.726550,HIGH
303,8.377555,51.444764,82.449742,1.172097,0.152341,1.326223,1.563590,1.404378,2.989927,HIGH
304,4.116524,52.383046,87.236185,1.272028,0.345993,1.622422,1.574452,1.643590,3.243919,HIGH


In [6]:
features = tf.keras.utils.normalize(df.iloc[:,0:-1], axis=-2, order=2).values
labels = df.iloc[:,-1].values
features

array([[ 0.08013911,  0.04834668,  0.07558855, ...,  0.05930982,
        -0.03576584,  0.00382095],
       [ 0.07663706,  0.04881112,  0.07475859, ...,  0.06064157,
        -0.02594362,  0.0113652 ],
       [ 0.03912381,  0.05210131,  0.08238617, ...,  0.08539712,
        -0.01216268,  0.03294639],
       ...,
       [ 0.02434114,  0.04996403,  0.06883846, ...,  0.01162529,
         0.00782945,  0.01088044],
       [ 0.01196064,  0.05087531,  0.07283473, ...,  0.01170605,
         0.00916307,  0.01180472],
       [ 0.02469881,  0.0597491 ,  0.08283392, ...,  0.04486752,
         0.01103964,  0.02960107]])

In [7]:
with open("models/tsla_18m_model.json", "r") as json_file:
    model = tf.keras.models.model_from_json(json_file.read())
    model.load_weights("models/tsla_18m_weights.h5")
    
    for i in list(range(-30, 0)):
        logits = model.predict(np.array([features[i]]))
   
        print(tf.nn.softmax(logits))
        print(labels[i])

tf.Tensor([[0.66216224 0.33783773]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[0.33481178 0.66518825]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[1.7397187e-13 1.0000000e+00]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[3.1599451e-15 1.0000000e+00]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[8.992552e-21 1.000000e+00]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[9.636597e-22 1.000000e+00]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[9.7878536e-32 1.0000000e+00]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[5.869048e-28 1.000000e+00]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[5.822781e-11 1.000000e+00]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[7.0032753e-09 1.0000000e+00]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[0.00290882 0.9970912 ]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[0.2215718  0.77842814]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[0.03909459 0.9609054 ]], shape=(1, 2), dtype=float32)
HIGH
tf.Tensor([[9.688749e-05 9.999031e-01